In [1]:
from neopipe import Task, Ok, Err

In [2]:
task = Task(lambda x: Ok(x + 1), retries=2)

In [3]:
task(2)

2024-06-25 22:16:13 - neopipe.task - INFO - Executing task <lambda> (UUID: 1a60307a-cb99-49d4-be10-15d76d17dc7e)
2024-06-25 22:16:13 - neopipe.task - INFO - Task <lambda> succeeded on attempt 1


Result(value=3, error=None)

In [4]:
def raise_exception(x: int):
    raise Exception(f"Exception {x}")

In [5]:
Task(raise_exception, retries=2)(3)

2024-06-25 22:16:37 - neopipe.task - INFO - Executing task raise_exception (UUID: 01310462-d81d-4070-92ab-dc83a6890f3f)
2024-06-25 22:16:37 - neopipe.task - ERROR - Task raise_exception exception on attempt 1: Exception 3
2024-06-25 22:16:38 - neopipe.task - ERROR - Task raise_exception exception on attempt 2: Exception 3


Result(value=None, error='Task raise_exception failed after 2 attempts: Exception 3')